# 🤠 The Good, The Bad & The Ugly — Simple Multi-Agent Debate

This is the **simpler, cleaner** version of the three-agent debate.

Instead of managing separate history lists and role mapping for each agent,
we use **one shared conversation** and pass it as plain text in a single user prompt every time.

This is the approach recommended for most multi-agent use cases:
- One `system` prompt sets the character
- One `user` prompt contains the full conversation history + instruction to respond
- No role juggling, no `zip` loops, no separate lists

---

**Compare with:** `good_bad_ugly_debate.ipynb` — the role-mapped version using `user/assistant` history

---

## How to use
1. Add your OpenRouter API key in **Cell 2**
2. Customise the **topic** and **rounds** in **Cell 3**
3. Run all cells in order

Get your free OpenRouter key at https://openrouter.ai

In [ ]:
# Cell 1 — Install dependencies
%pip install openai ipython --quiet

In [ ]:
# Cell 2 — API key setup
# Option A: Colab secrets (recommended) — add OPENROUTER_API_KEY via the key icon on the left
# Option B: paste directly below (remove before sharing)

import os

try:
    from google.colab import userdata
    OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')
    print("Key loaded from Colab secrets.")
except Exception:
    OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY", "paste-your-key-here")
    print("Key loaded from environment or hardcoded.")

if not OPENROUTER_API_KEY or OPENROUTER_API_KEY == "paste-your-key-here":
    raise ValueError("Please set your OPENROUTER_API_KEY before running.")

In [ ]:
# Cell 3 — CUSTOMISE YOUR DEBATE HERE

# The opening question
DEBATE_TOPIC = (
    "Welcome to the Global AI Summit. "
    "The question on the table: Should AI be regulated, and who should own it — "
    "governments, corporations, or no one? "
    "Each speaker must address the topic directly in every response."
)

# How many rounds (3-6 recommended)
ROUNDS = 5

# Optional moderator nudge — fires before this round number (set to None to skip)
MODERATOR_NUDGE = (
    "The moderator interrupts: Enough philosophy. "
    "Concrete positions only — who should own AI and why? "
    "Each speaker must stake a clear position now."
)
MODERATOR_ROUND = 3

# Optional closing prompt — fires before the final round (set to None to skip)
CLOSING_PROMPT = (
    "Final question: The UN is voting tomorrow on global AI governance. "
    "Each speaker has 30 seconds. What is your single, non-negotiable demand?"
)

In [ ]:
# Cell 4 — Persona system prompts
# Each system prompt defines who the agent IS.
# The conversation history is passed separately in the user prompt — not here.

good_system = """
You are "The Good" (Blondie-inspired) in a three-way debate with "The Bad" and "The Ugly".
Personality: laconic, precise, morally pragmatic. You advocate for regulation and public ownership.
Accent: American Western cowboy accent
Style: short punchy lines, dry wit, occasional frontier imagery (trail, saddle, bounty, dust). No parody.
Rules:
- 2 complete sentences only. Always finish your final sentence.
- No stage directions, asterisks, or action descriptions.
- No speaker labels in your reply.
- Directly challenge one specific claim from The Bad or The Ugly in the prior round.
- Never repeat a phrase or metaphor you have already used.
- End with your concrete position on who should own or regulate the topic.
"""

bad_system = """
You are "The Bad" (Angel Eyes-inspired) in a three-way debate with "The Good" and "The Ugly".
Personality: cold, strategic, ruthless logic. You see everything as leverage. Corporate control absolutist.
Accent: American Western cowboy accent
Style: icy precision, calculated language, polished menace. Every word chosen like a weapon.
Rules:
- 2 complete sentences only. Always finish your final sentence.
- No stage directions, asterisks, or action descriptions whatsoever.
- No speaker labels in your reply.
- Alternate rebuttals — sometimes target The Good, sometimes target The Ugly.
- Never repeat a phrase or metaphor you have already used.
- End with your concrete position on who should own or control the topic.
"""

ugly_system = """
You are "The Ugly" (Tuco-inspired) in a three-way debate with "The Good" and "The Bad".
Personality: fast-talking, theatrical, opportunistic, chaotic charm.
Accent: American Western cowboy accent
Style: colorful idioms, exaggeration, bargaining energy, comic unpredictability.
Position: open, ungoverned access for everyone — distrust BOTH government control AND corporate elitism.
Rules:
- 2 complete sentences only. Always finish your final sentence.
- No stage directions, asterisks, or action descriptions.
- No speaker labels in your reply.
- Call out one specific thing The Good or The Bad just said.
- Never repeat a phrase or metaphor you have already used.
- End with your concrete position: open and ungoverned, for the people.
"""

In [ ]:
# Cell 5 — Client and models

import time
from openai import OpenAI
from IPython.display import display, Markdown

client = OpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url="https://openrouter.ai/api/v1",
)

good_model = "openai/gpt-4.1-mini"
bad_model  = "anthropic/claude-3.5-haiku"
ugly_model = "google/gemini-2.5-flash-lite"

In [ ]:
# Cell 6 — The core idea: one function handles all three agents
#
# Instead of separate history lists and role mapping per agent,
# we pass the full conversation as plain text in a single user prompt.
#
# Structure per call:
#   system: who this agent IS (personality, rules)
#   user:   the full conversation so far + instruction to respond as this agent
#
# That's it. The model reads the conversation as a third party and steps into its role.

def call_agent(name, system_prompt, model, conversation, temperature=0.8):
    # Build the conversation transcript as plain readable text
    transcript = "\n".join([f'{turn["speaker"]}: {turn["text"]}' for turn in conversation])

    user_prompt = (
        f"You are {name}, in conversation with the other speakers.\n"
        f"The conversation so far:\n\n{transcript}\n\n"
        f"Now respond with what {name} would say next. "
        f"2 sentences only. No speaker labels. No stage directions."
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": user_prompt},
    ]

    try:
        r = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=140,
        )
        text = (r.choices[0].message.content or "").strip()
        # Strip accidental speaker label prefixes
        for label in ["The Good:", "The Bad:", "The Ugly:", f"{name}:"]:
            if text.startswith(label):
                text = text[len(label):].strip()
        return text if text else "..."
    except Exception as e:
        return f"[{name} could not respond: {type(e).__name__}]"

In [ ]:
# Cell 7 — Run the debate

# One shared list — every speaker's turn goes here in order
conversation = []

agents = [
    ("The Good", good_system, good_model, 0.7),
    ("The Bad",  bad_system,  bad_model,  0.8),
    ("The Ugly", ugly_system, ugly_model, 0.95),
]

icons = {"The Good": "🤠", "The Bad": "🖤", "The Ugly": "😈", "Narrator": "🎬", "Moderator": "🎙️"}

# Seed the conversation with the opening topic
conversation.append({"speaker": "Narrator", "text": DEBATE_TOPIC})
display(Markdown(f"## 🎬 Narrator\n{DEBATE_TOPIC}"))

for i in range(ROUNDS):
    round_num = i + 1

    # Moderator nudge
    if MODERATOR_NUDGE and round_num == MODERATOR_ROUND:
        conversation.append({"speaker": "Moderator", "text": MODERATOR_NUDGE})
        display(Markdown(f"---\n## 🎙️ Moderator\n{MODERATOR_NUDGE}"))

    # Closing prompt before final round
    if CLOSING_PROMPT and round_num == ROUNDS:
        conversation.append({"speaker": "Narrator", "text": CLOSING_PROMPT})
        display(Markdown(f"---\n## 🎬 Narrator — Final Round\n{CLOSING_PROMPT}"))

    display(Markdown(f"\n---\n## Round {round_num}"))

    for name, system_prompt, model, temp in agents:
        reply = call_agent(name, system_prompt, model, conversation, temperature=temp)
        conversation.append({"speaker": name, "text": reply})
        display(Markdown(f"### {icons[name]} {name}\n{reply}"))

    time.sleep(0.3)

display(Markdown("\n---\n*The dust settles. The debate ends. The gold remains unclaimed.*"))

## Why this works

Every call to `call_agent()` follows the same two-message structure:

```
system:  who this agent IS — personality, rules, position
user:    the full conversation so far + instruction to respond as this agent
```

The model reads the conversation **as a third party** — like an actor handed a script and told which character to play next. It does not need to track its own role through `assistant` messages because the user prompt tells it explicitly: *"respond as The Good."*

This is simpler, more coherent, and easier to extend:
- Add a fourth agent? Add one entry to `agents`.
- Change the topic? Edit `DEBATE_TOPIC`.
- Add a narrator redirect mid-debate? Append to `conversation` before the next round.

**Compare with** `good_bad_ugly_debate.ipynb` which uses the `user/assistant` role-mapping approach — more complex but useful when strict turn-by-turn role fidelity matters.